In [2]:
! pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 915.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 514.8 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tqdm.auto import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [7]:
# one-step sarima forecast
def sarima_forecast(data, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(
        data,
        order=order,
        seasonal_order=sorder,
        trend=trend,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    # fit model
    model_fit = model.fit(disp=True)
    return model_fit

In [3]:
# Avarage temp

In [13]:
train = pd.read_csv('../monica/monica_weather_2011-2016.csv', 
                      index_col = ['de-date'], 
                      parse_dates = ['de-date'], 
                      dayfirst = True)
train = train.asfreq('d')

test = pd.read_csv('../monica/reanalysis_2017.csv')

In [3]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [15]:
test.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
0,01.01.2017,-3.85,-6.03,-1.72,3.96,2.14,0.90,98.50
1,02.01.2017,-1.89,-3.75,-0.85,5.43,1.92,0.57,96.75
2,03.01.2017,-3.72,-9.02,-1.72,5.07,2.04,0.90,95.38
3,04.01.2017,-6.60,-9.16,-5.06,5.27,3.57,3.88,97.56
4,05.01.2017,-4.36,-6.16,-2.20,3.25,3.15,1.17,97.38


In [16]:
tavg = sarima_forecast(data=train.tavg, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmin = sarima_forecast(data=train.tmin, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmax = sarima_forecast(data=train.tmax, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
wind = sarima_forecast(data=train.wind, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
globrad = sarima_forecast(data=train.globrad, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
precip = sarima_forecast(data=train.precip, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
relhumid = sarima_forecast(data=train.relhumid, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])

In [17]:
tavg_preds = tavg.predict(start=2*365, end=3*365-1)
tmin_preds = tmin.predict(start=2*365, end=3*365-1)
tmax_preds = tmax.predict(start=2*365, end=3*365-1)
wind_preds = wind.predict(start=2*365, end=3*365-1)
globrad_preds = globrad.predict(start=2*365, end=3*365-1)
precip_preds = precip.predict(start=2*365, end=3*365-1)
relhumid_preds = relhumid.predict(start=2*365, end=3*365-1)

In [18]:
len(tavg_preds)

365

In [19]:
test.tavg = tavg_preds.to_numpy()
test.tmin = tmin_preds.to_numpy()
test.tmax = tmax_preds.to_numpy()
test.wind = wind_preds.to_numpy()
test.globrad = globrad_preds.to_numpy()
test.precip = precip_preds.to_numpy()
test.relhumid = relhumid_preds.to_numpy()

In [20]:
test.to_csv('../monica/sarimax_2017.csv', index=False)

# WOFOST

In [18]:
train = pd.read_csv('../wofost/reanalysis_2011_2016.csv', 
                      index_col = ['DAY'], 
                      parse_dates = ['DAY'], 
                      dayfirst = True)
train = train.asfreq('d')

test = pd.read_csv('../wofost/reanalysis_2017.csv')

In [19]:
test.head()

,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH
0,20170101,2140.0,-6.31,-1.60,0.443766,3.77,0.79,NaN
1,20170102,1920.0,-4.31,-0.91,0.491824,5.28,0.35,NaN
2,20170103,2040.0,-6.40,-1.92,0.430543,5.09,0.74,NaN
3,20170104,3570.0,-7.85,-4.60,0.360791,5.46,2.64,NaN
4,20170105,3150.0,-5.21,-1.74,0.434473,3.29,1.52,NaN


In [20]:
irrad = sarima_forecast(data=train.IRRAD, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmin = sarima_forecast(data=train.TMIN, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
tmax = sarima_forecast(data=train.TMAX, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
wind = sarima_forecast(data=train.WIND, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
vap = sarima_forecast(data=train.VAP, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])
rain = sarima_forecast(data=train.RAIN, config=[(0, 0, 2), (2, 0, 2, 12), 'c'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.42398D+01    |proj g|=  2.75923D+01


 This problem is unconstrained.



At iterate    1    f=  1.00343D+01    |proj g|=  2.41966D-01

At iterate    2    f=  1.00168D+01    |proj g|=  2.12506D-01

At iterate    3    f=  9.96796D+00    |proj g|=  3.88202D-02

At iterate    4    f=  9.96677D+00    |proj g|=  3.02091D-02

At iterate    5    f=  9.96468D+00    |proj g|=  8.31752D-03

At iterate    6    f=  9.96446D+00    |proj g|=  7.08040D-03

At iterate    7    f=  9.96382D+00    |proj g|=  1.17681D-02

At iterate    8    f=  9.96227D+00    |proj g|=  1.90516D-02

At iterate    9    f=  9.96067D+00    |proj g|=  2.91333D-02

At iterate   10    f=  9.96042D+00    |proj g|=  2.28457D-02

At iterate   11    f=  9.96007D+00    |proj g|=  1.85884D-02

At iterate   12    f=  9.95822D+00    |proj g|=  1.00798D-02

At iterate   13    f=  9.95759D+00    |proj g|=  4.97195D-03

At iterate   14    f=  9.95749D+00    |proj g|=  2.86196D-03

At iterate   15    f=  9.95747D+00    |proj g|=  9.63086D-04

At iterate   16    f=  9.95747D+00    |proj g|=  8.45272D-05

At iter

 This problem is unconstrained.



At iterate    1    f=  3.29026D+00    |proj g|=  1.56537D-01

At iterate    2    f=  3.25211D+00    |proj g|=  8.26985D-02

At iterate    3    f=  3.23866D+00    |proj g|=  8.46881D-02

At iterate    4    f=  3.16831D+00    |proj g|=  9.05954D-02

At iterate    5    f=  2.96635D+00    |proj g|=  2.76415D-01

At iterate    6    f=  2.83774D+00    |proj g|=  3.62193D-01

At iterate    7    f=  2.81978D+00    |proj g|=  3.71497D-01

At iterate    8    f=  2.73340D+00    |proj g|=  3.64416D-01

At iterate    9    f=  2.70062D+00    |proj g|=  1.73675D-01

At iterate   10    f=  2.69674D+00    |proj g|=  7.84520D-02

At iterate   11    f=  2.68209D+00    |proj g|=  6.89200D-02

At iterate   12    f=  2.67891D+00    |proj g|=  4.98977D-02

At iterate   13    f=  2.67766D+00    |proj g|=  1.90307D-02

At iterate   14    f=  2.67718D+00    |proj g|=  1.20979D-02

At iterate   15    f=  2.67678D+00    |proj g|=  1.23971D-02

At iterate   16    f=  2.67594D+00    |proj g|=  1.37496D-02

At iter

...

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.85506D+01    |proj g|=  8.24662D+02

At iterate    1    f=  2.02290D+00    |proj g|=  9.83090D-01

At iterate    2    f=  1.88300D+00    |proj g|=  7.82306D-01

At iterate    3    f=  1.62099D+00    |proj g|=  1.99425D-01

At iterate    4    f=  1.59614D+00    |proj g|=  1.34938D-01

At iterate    5    f=  1.58961D+00    |proj g|=  1.02368D-01

At iterate    6    f=  1.58267D+00    |proj g|=  1.18478D-01

At iterate    7    f=  1.58046D+00    |proj g|=  1.18731D-01

At iterate    8    f=  1.57705D+00    |proj g|=  1.05480D-01

At iterate    9    f=  1.56986D+00    |proj g|=  9.20712D-02

At iterate   10    f=  1.56488D+00    |proj g|=  8.04641D-02

At iterate   11    f=  1.56035D+00    |proj g|=  4.58712D-02

At iterate   12    f=  1.55811D+00    |proj g|=  1.12656D-02

At iterate   13    f=  1.5

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.56494D+00    |proj g|=  4.95342D+00

At iterate    1    f=  7.91985D-01    |proj g|=  1.07656D+00

At iterate    2    f= -1.17650D-01    |proj g|=  8.94223D-01

At iterate    3    f= -1.21285D-01    |proj g|=  6.84543D-01

At iterate    4    f= -1.39075D-01    |proj g|=  2.34736D-01

At iterate    5    f= -1.58443D-01    |proj g|=  1.12942D+00

At iterate    6    f= -1.70967D-01    |proj g|=  3.71708D-01

At iterate    7    f= -1.78481D-01    |proj g|=  1.60862D-01

At iterate    8    f= -1.82992D-01    |proj g|=  1.33707D-01

At iterate    9    f= -1.97002D-01    |proj g|=  3.10321D-01

At iterate   10    f= -2.07885D-01    |proj g|=  4.32620D-01

At iterate   11    f= -2.21481D-01    |proj g|=  3.42776D-01

At iterate   12    f= -2.33173D-01    |proj g|=  3.53886D-01

At iterate   13    f= -2.3

 This problem is unconstrained.



At iterate    1    f=  2.78583D+00    |proj g|=  3.82105D-01

At iterate    2    f=  2.73625D+00    |proj g|=  2.77003D-01

At iterate    3    f=  2.64524D+00    |proj g|=  6.97942D-02

At iterate    4    f=  2.64225D+00    |proj g|=  2.68374D-02

At iterate    5    f=  2.64184D+00    |proj g|=  1.61590D-02

At iterate    6    f=  2.64122D+00    |proj g|=  1.77168D-02

At iterate    7    f=  2.64037D+00    |proj g|=  2.56478D-02

At iterate    8    f=  2.63868D+00    |proj g|=  3.19486D-02

At iterate    9    f=  2.63723D+00    |proj g|=  2.42759D-02

At iterate   10    f=  2.63655D+00    |proj g|=  1.30857D-02

At iterate   11    f=  2.63638D+00    |proj g|=  3.10946D-03

At iterate   12    f=  2.63636D+00    |proj g|=  1.96353D-03

At iterate   13    f=  2.63636D+00    |proj g|=  1.41152D-03

At iterate   14    f=  2.63634D+00    |proj g|=  1.60573D-03

At iterate   15    f=  2.63634D+00    |proj g|=  1.81399D-03

At iterate   16    f=  2.63632D+00    |proj g|=  1.91096D-03

At iter

In [21]:
irrad_preds = irrad.predict(start=2*365, end=3*365 - 1)
tmin_preds = tmin.predict(start=2*365, end=3*365 - 1)
tmax_preds = tmax.predict(start=2*365, end=3*365 - 1)
wind_preds = wind.predict(start=2*365, end=3*365 - 1)
vap_preds = vap.predict(start=2*365, end=3*365 - 1)
rain_preds = rain.predict(start=2*365, end=3*365 - 1)

In [22]:
test.IRRAD = irrad_preds.to_numpy()
test.TMIN = tmin_preds.to_numpy()
test.TMAX = tmax_preds.to_numpy()
test.WIND = wind_preds.to_numpy()
test.VAP = vap_preds.to_numpy()
test.RAIN = rain_preds.to_numpy()

In [23]:
test.to_csv('../wofost/sarimax_2017.csv', index=False)